In [ ]:
from obspy.clients.fdsn import Client
client = Client("iris")
print(client)

In [ ]:
client.services

In [ ]:
import requests
url='https://service.iris.edu/irisws/fedcatalog/1/'
r=requests.get(url + "query", params={"net":"A*","sta":"OK*","cha":"*HZ"}, verify=False)


In [ ]:
r

In [ ]:
print(r.content)

In [ ]:
rtext= '''
    sta=A*
    minlat=34
    maxlat=38
    cha=?HZ
    starttime=2015-01-01
    endtime=2016-01-02'''
r = requests.post(url+"query",data='includeoverlaps=true\nA* OR* * BHZ 2015-01-01T00:00:00 2016-01-02T00:00:00', verify=False)
print(r)
print(r.content)

In [1]:
class RequestLine(object):
    def __init__(self, line):
        self.line = line.strip()
        
    def is_empty(self):
        return self.line == ""

    def is_datacenter(self):
        return self.line.startswith('DATACENTER=')

    def is_param(self):
        # true for datacenter, services, and parameter_list
        return '=' in self.line

    def is_request(self):
        return len(self.line.split()) == 6 # and test field values?

    def is_service(self):
        # parse param_name
        return self.is_param() and self.line.split("=")[0].isupper() and not self.is_datacenter()

    def __repr__(self):
        return self.line

    def __str__(self):
        return self.line


In [2]:

class ParserState(object):
    '''
    parser states: PREPARSE, PARAMLIST, EMPTY_LINE, DATACENTER, SERVICE, REQUEST, DONE
    PREPARSE -> [PARAMLIST | EMPTY_LINE | DATACENTER]
    PARAMLIST -> [PARAMLIST | EMPTY_LINE]
    EMPTY_LINE -> [EMPTY_LINE | DATACENTER | DONE]
    DATACENTER -> [SERVICE]
    SERVICE -> [SERVICE | REQUEST]
    REQUEST -> [REQUEST | EMPTY_LINE | DONE ]
    '''

    @staticmethod
    def parse(line, this_response):
        '''abstract'''
        raise NotImplementedError, "ParserState.parse()"

    @staticmethod
    def next(line):
        '''abstract'''
        raise NotImplementedError, "ParserState.next()"

class PreParse(ParserState):
    '''Initial ParserState'''

    @staticmethod
    def parse(line, this_response):
        return this_response

    @staticmethod
    def next(line):
        if line.is_empty():
            return EmptyItem #EMPTY_LINE
        elif line.is_datacenter():
            return DatacenterItem #DATACENTER
        elif line.is_param():
            return ParameterItem #PARAMLIST
        else:
            return ParserState

class ParameterItem(ParserState):
    '''handle a parameter'''

    @staticmethod
    def parse(line, this_response):
        '''Parse: param=value'''
        this_response.add_common_parameters(line)
        return this_response

    def next(self, line):
        if line.is_empty():
            return EmptyItem() #EMPTY_LINE
        elif line.is_param():
            return self
        else:
            raise RuntimeError, "Parameter should be followed by another parameter or an empty line"

class EmptyItem(ParserState):
    '''handle an empty line'''

    @staticmethod
    def parse(line, this_response):
        return this_response
    
    @staticmethod
    def next(line):
        if line.is_empty():
            return EmptyItem #no state change
        elif line.is_datacenter():
            return DatacenterItem #DATACENTER
        else:
            raise RuntimeError, "expected either a DATACENTER or another empty line"

class DatacenterItem(ParserState):
    '''handle data center'''

    @staticmethod
    def parse(line, this_response):
        '''Parse: DATACENTER=id,http://url...'''
        _, rest = str(line).split('=')
        active_id, url = rest.split(',')
        this_response = FederatedResponseParser.new_federated_response(active_id)
        print("new response", this_response)
        this_response.add_service("DATACENTER", url)
        return this_response

    @staticmethod
    def next(line):
        if line.is_service():
            return ServiceItem
        else:
            raise RuntimeError, "DATACENTER line should be followed by a service"

class ServiceItem(ParserState):
    '''handle service description'''

    @staticmethod
    def parse(line, this_response):
        '''Parse: SERICENAME=http://service.url/'''
        svc_name, url = str(line).split('=')
        this_response.add_service(svc_name, url)
        return this_response

    @staticmethod
    def next(line):
        if line.is_service():
            return ServiceItem
        elif line.is_request():
            return RequestItem
        else:
            raise RuntimeError, "Service desc. should be followed by a request or another service"

class RequestItem(ParserState):
    '''handle request lines'''

    @staticmethod
    def parse(line, this_response):
        '''Parse: NT STA LC CHA YYYY-MM-DDThh:mm:ss YY-MM-DDThh:mm:ss'''
        this_response.add_request_lines(line)
        return this_response

    @staticmethod
    def next(line):
        if line.is_request():
            return RequestItem
        elif line.is_empty():
            return EmptyItem
        else:
            raise RuntimeError, "Requests should be followed by another request or an empty line"

In [3]:

class FederatedResponse(object):
    '''
    >>> fed_resp = FederatedResponse("IRISDMC")
    >>> fed_resp.add_common_parameters(["lat=50","lon=20","level=cha"])
    >>> fed_resp.add_service("STATIONSERVICE","http://service.iris.edu/fdsnws/station/1/")
    >>> fed_resp.add_request_line("AI ORCD -- BHZ 2015-01-01T00:00:00 2016-01-02T00:00:00")
    >>> fed_resp.add_request_line("AI ORCD 04 BHZ 2015-01-01T00:00:00 2016-01-02T00:00:00")
    >>> print(fed_resp.request_text("STATIONSERVICE"))

    level=cha
    AI ORCD -- BHZ 2015-01-01T00:00:00 2016-01-02T00:00:00
    AI ORCD 04 BHZ 2015-01-01T00:00:00 2016-01-02T00:00:00
    '''

    ok_parameters = {"DATASELECTSERVICE":["longestonly"],
                        "STATIONSERVICE":["level"]}

    def __init__(self, datacenter_id):
        self.datacenter_id = datacenter_id
        self.common_parameters = []
        self.services = {}
        self.request_lines = []

    def add_service(self, service_name, service_url):
        self.services[service_name] = service_url

    def add_common_parameters(self, common_parameters):
        if isinstance(common_parameters, str):
            self.common_parameters.append(common_parameters)
        elif isinstance(common_parameters, RequestLine):
            self.request_lines.append(str(common_parameters))
        else:
            self.common_parameters.extend(common_parameters)

    def add_request_lines(self, request_lines):
        if isinstance(request_lines, str):
            self.request_lines.append(request_lines)
        elif isinstance(request_lines, RequestLine):
            self.request_lines.append(str(request_lines))
        else:
            self.request_lines.extend(request_lines)

    def add_request_line(self, request_line):
        self.request_lines.append(request_line)

    def request_text(self, target_service):
        reply = self.selected_common_parameters(target_service)
        reply.extend(self.request_lines)
        return "\n".join(reply)

    def selected_common_parameters(self, target_service):
        reply = []
        for good in FederatedResponse.ok_parameters[target_service]:
            reply.extend([c for c in self.common_parameters if c.startswith(good + "=")])
        return reply
    
    def __repr__(self):
        return self.datacenter_id + "\n" + self.request_text("STATIONSERVICE")

In [17]:
class FederatedResponseParser(object):
    '''Iterate through stream, returning FederatedResponse objects for each datacenter'''
    def __init__(self, stream_iterator):
        self.stream_iterator = stream_iterator() # stream_iterator feeds us line by line
        self.state = PreParse
        self.n_datacenters = 0
        self.fed_req = None
        self.line = None

    def __iter__(self):
        return self
    
    def next(self):
        any_request_was_processed = False
        print("A Next...", self.state)
        if self.line is not None:
            print("returned to NEXT")
            self.fed_req = self.state.parse(self.line, self.fed_req) #left before processing
            self.line = None
        print(self.state)
        for self.line in self.stream_iterator:
            self.line = RequestLine(self.line)
            self.state = self.state.next(self.line)
            print(self.state)
            if any_request_was_processed and (self.state is not RequestItem):
                return self.fed_req
            self.fed_req = self.state.parse(self.line, self.fed_req)
            if self.state == RequestItem:
                any_request_was_processed = True
            
        raise StopIteration
        #
        #if any_request_was_processed:
        #    print("G we processed at least one request", self.state)
        #    return self.fed_req
        #else:
        #    print("H no requests were processed", self.state)
        #    raise StopIteration
            
    __next__ = next

    @staticmethod
    def new_federated_response(ds_id):
        return FederatedResponse(ds_id)

In [25]:
import requests
url='https://service.iris.edu/irisws/fedcatalog/1/'
r=requests.get(url + "query", params={"net":"A*","sta":"OR*","cha":"*HZ","includeoverlaps":"true"}, verify=False)
frp = FederatedResponseParser(r.iter_lines)
print(r.content)
for n in frp:
    print(n)
    #print(n.request_text("STATIONSERVICE"))


/Users/celso/anaconda/lib/python2.7/site-packages/requests/packages/urllib3/connectionpool.py:843: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)



DATACENTER=IRISDMC,http://ds.iris.edu
DATASELECTSERVICE=http://service.iris.edu/fdsnws/dataselect/1/
STATIONSERVICE=http://service.iris.edu/fdsnws/station/1/
EVENTSERVICE=http://service.iris.edu/fdsnws/event/1/
SACPZSERVICE=http://service.iris.edu/irisws/sacpz/1/
RESPSERVICE=http://service.iris.edu/irisws/resp/1/
AI ORCD -- BHZ 1997-03-01T00:00:00 2599-12-31T23:59:59
AI ORCD 02 HHZ 2004-03-20T00:00:00 2599-12-31T23:59:59
AI ORCD 04 BHZ 2004-03-20T00:00:00 2599-12-31T23:59:59
AI ORCD 06 MHZ 2004-03-20T00:00:00 2599-12-31T23:59:59

DATACENTER=ORFEUS,http://www.orfeus-eu.org
DATASELECTSERVICE=http://www.orfeus-eu.org/fdsnws/dataselect/1/
STATIONSERVICE=http://www.orfeus-eu.org/fdsnws/station/1/
AI ORCD -- BHZ 1997-03-01T00:00:00 2599-12-31T23:59:59



('A Next...', <class '__main__.PreParse'>)
<class '__main__.PreParse'>
<class '__main__.EmptyItem'>
<class '__main__.DatacenterItem'>
('new response', IRISDMC
)
<class '__main__.ServiceItem'>
<class '__main__.ServiceItem'>
<class '__main__.

In [9]:
fed_resp = FederatedResponse("IRISDMC")
fed_resp.add_common_parameters("dummy=dummy")
fed_resp.add_common_parameters(["lat=50","lon=20","level=cha"])
fed_resp.add_service("STATIONSERVICE","http://service.iris.edu/fdsnws/station/1/")
fed_resp.add_request_lines("AI ORCD -- BHZ 2015-01-01T00:00:00 2016-01-02T00:00:00")
fed_resp.add_request_lines(["AI ORCD 04 BHZ 2015-01-01T00:00:00 2016-01-02T00:00:00"])
print(fed_resp.request_text("STATIONSERVICE"))

level=cha
AI ORCD -- BHZ 2015-01-01T00:00:00 2016-01-02T00:00:00
AI ORCD 04 BHZ 2015-01-01T00:00:00 2016-01-02T00:00:00


In [ ]:
frp = FederatedResponseParser(r.iter_lines)
print("frp" , frp.__class__)

In [ ]:
?frp

In [ ]:
print(r.content)

In [ ]:
n.__class__